In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain_community.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough

In [2]:
# 문서 로드
loader = TextLoader("murder_mystery.txt")
documents = loader.load()

In [3]:
# 문서 분할 설정 수정
text_splitter = CharacterTextSplitter(
    chunk_size=2048,
    chunk_overlap=200,
    separator="\n\n"
)

splits = text_splitter.split_documents(documents)

Created a chunk of size 2313, which is longer than the specified 2048
Created a chunk of size 2247, which is longer than the specified 2048


In [4]:
# 임베딩 생성 및 캐시 설정
underlying_embeddings = OpenAIEmbeddings()
fs = LocalFileStore("./cache/")
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings,
    fs,
    namespace=underlying_embeddings.model
)

In [5]:
# FAISS 검색 설정
vectorstore = FAISS.from_documents(splits, cached_embedder)

In [6]:
# 검색 설정
retriever = vectorstore.as_retriever(
    search_kwargs={
        "k": 6,  # 검색 결과 수 조정
        "fetch_k": 10  # 필터링 전 검색할 문서 수
    }
)

In [7]:
# 프롬프트 템플릿
prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 주어진 텍스트에 대해서 분석하는 문학 전문가입니다. 
    주어진 context를 바탕으로 질문에 정확하고 상세하게 답변해주세요.
    
    답변 시 주의사항:
    1. context에서 찾을 수 있는 구체적인 내용을 포함해주세요
    2. 등장인물의 행동이나 대화를 인용하면서 설명해주세요
    3. 답을 모르는 경우, 명확히 모른다고 답변해주세요
    4. 답변의 근거가 되는 내용을 context에서 찾아 설명해주세요
    
    Context: {context}"""),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}")
])

In [8]:
# 메모리 설정
memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="history"
)

/var/folders/7p/6bdnyyl97jq1l0vwbrmy3k600000gn/T/ipykernel_17366/851550066.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [9]:
# LLM 설정
model = ChatOpenAI(
    model="gpt-4o-mini-2024-07-18",
    temperature=0.1
)

In [10]:
# RAG 체인 구성
chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
        "history": lambda x: memory.load_memory_variables({})["history"]
    }
    | prompt
    | model
)

In [11]:
# 질문 함수 정의
def ask_question(question: str):
    response = chain.invoke(question)
    # 대화 저장
    memory.save_context(
        {"input": question},
        {"output": response.content}
    )
    return response.content

In [12]:
# 질문하기
questions = [
    "Aaronson은 유죄인가요?",
    "그가 테이블에 어떤 메시지를 썼나요?",
    "Julia 는 누구인가요?"
]

for question in questions:
    print(f"\nQuestion: {question}")
    print(f"Answer: {ask_question(question)}\n")


Question: Aaronson은 유죄인가요?
Answer: 주어진 context에서는 Aaronson이 유죄라는 주장이 있습니다. Winston은 "Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with"라고 언급하며, 그들이 유죄라는 것을 받아들이고 있습니다. 그러나 Winston은 나중에 "He had never seen the photograph that disproved their guilt. It had never existed, he had invented it"라고 회상하며, 자신의 기억이 왜곡되었음을 깨닫습니다. 이는 Aaronson이 실제로 유죄가 아닐 수도 있음을 암시합니다. 따라서, Aaronson의 유죄 여부는 확실하지 않으며, Winston의 인식과 당국의 주장 사이에 모순이 존재합니다.


Question: 그가 테이블에 어떤 메시지를 썼나요?
Answer: Winston은 테이블에 다음과 같은 메시지를 썼습니다:

1. "FREEDOM IS SLAVERY"
2. "TWO AND TWO MAKE FIVE"
3. "GOD IS POWER"

이 메시지들은 Winston이 당국의 사상에 순응하려고 노력하는 과정에서 나타나는 그의 생각을 반영합니다. 그는 처음에 "FREEDOM IS SLAVERY"를 대문자로 썼고, 그 다음에 "TWO AND TWO MAKE FIVE"를 적었습니다. 그러나 그가 다음에 적으려 할 때, 그의 마음이 뭔가를 회피하는 듯한 느낌을 받았고, 결국 "GOD IS POWER"를 적었습니다. 이러한 과정은 그가 당국의 세뇌와 통제에 굴복하고 있음을 보여줍니다.


Question: Julia 는 누구인가요?
Answer: Julia는 Winston의 사랑하는 사람으로, 그들의 관계는 주어진 context에서 중요한 역할을 합니다. Winston은 Julia를 깊이 사랑하고 있으며, 그녀와의 관계는 그가 당국에 맞서 싸우는 데 